<a href="https://colab.research.google.com/github/JazmineOrtizMarin/Simulaci-n-2/blob/main/MM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

| no. Paquete | T. llegada | T. salida | Tiempo de arribo | Inicio de servicio | Hora de salida | Tiempo en fila | Tiempo en el sistema |
|-------------|------------|-----------|-------------------|---------------------|-----------------|----------------|------------------------|
| 1           | 2          | 12        | 2                 | 2                   | 14              | 0              | 12                     |
| 2           | 5          | 10        | 7                 | 14                  | 24              | 7              | 17                     |
| 3           | 1          | 16        | 8                 | 24                  | 40              | 16             | 32                     |
| 4           | 4          | 9         | 12                | 40                  | 49              | 28             | 37                     |
| 5           | 1          | 10        | 13                | 49                  | 59              | 36             | 46                     |
| 6           | 3          | 13        | 16                | 59                  | 72              | 43             | 56                     |
| 7           | 3          | 17        | 19                | 72                  | 89              | 53             | 70                     |
| 8           | 2          | 10        | 21                | 89                  | 99              | 68             | 78                     |
| 9           | 4          | 8         | 25                | 99                  | 107             | 74             | 82                     |
| 10          | 5          | 12        | 30                | 107                 | 119             | 77             | 89                     |


In [10]:
import random

In [11]:
# ----- Parámetros del modelo M/M/1 -----
lambd = 0.2   # tasa de llegadas (λ)      -> media interarribo = 1/λ
mu    = 0.3   # tasa de servicio (μ)      -> media servicio     = 1/μ
n_paquetes = 10

random.seed(42)

In [12]:
# Listas para ir guardando todo
no_paquete      = []
t_llegada       = []  # interarribos
t_servicio      = []  # tiempos de servicio
t_arribo        = []  # tiempo de arribo (acumulado)
inicio_servicio = []
hora_salida     = []
t_fila          = []
t_sistema       = []

tiempo_actual_llegadas = 0.0
tiempo_salida_anterior = 0.0  # hora en que terminó el paquete anterior

for i in range(1, n_paquetes + 1):
    # --- generar tiempos aleatorios ---
    interarribo = random.expovariate(lambd)  # Exp(λ)
    servicio    = random.expovariate(mu)     # Exp(μ)

    # --- actualizar tiempos de arribo ---
    tiempo_actual_llegadas += interarribo
    llegada = tiempo_actual_llegadas

    # --- lógica del servidor (FIFO, un solo servidor) ---
    inicio = max(llegada, tiempo_salida_anterior)
    salida = inicio + servicio

    # --- métricas de desempeño ---
    espera_fila   = inicio - llegada
    en_sistema    = salida - llegada

    # --- guardar todo ---
    no_paquete.append(i)
    t_llegada.append(interarribo)
    t_servicio.append(servicio)
    t_arribo.append(llegada)
    inicio_servicio.append(inicio)
    hora_salida.append(salida)
    t_fila.append(espera_fila)
    t_sistema.append(en_sistema)

    tiempo_salida_anterior = salida  # actualizar para el siguiente

In [13]:
# ----- impresión de la tabla -----
print(
    f"{'no':>2} {'T_llegada':>10} {'T_salida':>10} {'T_arribo':>10} "
    f"{'Inicio_serv':>12} {'Hora_salida':>12} {'T_fila':>10} {'T_sistema':>12}"
)
for i in range(n_paquetes):
    print(
        f"{no_paquete[i]:>2} "
        f"{t_llegada[i]:>10.2f} "
        f"{t_servicio[i]:>10.2f} "
        f"{t_arribo[i]:>10.2f} "
        f"{inicio_servicio[i]:>12.2f} "
        f"{hora_salida[i]:>12.2f} "
        f"{t_fila[i]:>10.2f} "
        f"{t_sistema[i]:>12.2f}"
    )

no  T_llegada   T_salida   T_arribo  Inicio_serv  Hora_salida     T_fila    T_sistema
 1       5.10       0.08       5.10         5.10         5.18       0.00         0.08
 2       1.61       0.84       6.71         6.71         7.55       0.00         0.84
 3       6.67       3.76      13.38        13.38        17.14       0.00         3.76
 4      11.14       0.30      24.51        24.51        24.82       0.00         0.30
 5       2.74       0.10      27.25        27.25        27.35       0.00         0.10
 6       1.23       2.35      28.49        28.49        30.83       0.00         2.35
 7       0.13       0.74      28.62        30.83        31.57       2.21         2.95
 8       5.25       2.62      33.87        33.87        36.49       0.00         2.62
 9       1.25       2.97      35.11        36.49        39.46       1.38         4.35
10       8.29       0.02      43.40        43.40        43.42       0.00         0.02
